In [4]:
from selenium import webdriver as wd
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd

In [5]:
#Variables
page_start = 1
page_finish = 21

In [6]:
from selenium import webdriver as wd
import csv
from bs4 import BeautifulSoup as bs

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&page{}&crid=1HT27OCBDLP2O&sprefix={}%2Caps%2C87&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
        
    global page_counter
    page_counter=1
    for page_counter in range(1,21):
        page_inserter = str(page_counter)
        url = template.format(search_term, page_inserter, search_term)
        page_counter += 1
        print(page_counter)
    return url

def extract_record(item):
    #Desc and URL
    atag = item.h2.a
    desc = atag.text.strip()
    url = 'https://www.amazon.com/' + atag.get('href')
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base','class': 's-underline-text'}).text
    except AttributeError:
        rating = ' '
        review_count = ' '
        
    result = (desc, price, rating, review_count, url)
    return result

def main(search_term):
    #WebDriver
    driver = wd.Chrome(r'Z:\Class Repo\Personal Practice\Web Scraping\chromedriver_win32\chromedriver.exe')
    url = 'https://www.amazon.com/'
    driver.get(url)
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,2):
        driver.get(url.format(page))
        soup = bs(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type':'s-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    #Save to csv
    with open('Amazon_Results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow({'Description', 'Price', 'Rating', 'Review Count', 'URL'})
        writer.writerows(records)
    
main('hoodie')

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
